<a href="https://colab.research.google.com/github/Cylick-d/A-Two-Stage-Parameter-Efficient-Fine-Tuning-Framework-LoRA-/blob/main/UNSW_capstone_loratest0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



如何**自己从 0 写出那 40 行 LoRA quickstart**
我们现在把两个实验彻底拆开。

---

# 实验 1：LoRA 挂载 + Forward

---

## 🎯 目标

验证三件事：

1. LoRA 能成功挂载到模型
2. 可训练参数比例显著下降
3. forward 能正常跑

---

## 🧠 整体逻辑结构

这个实验只有 5 个步骤：

```
1. 装库
2. 加载模型 & tokenizer
3. 定义 LoRA 配置
4. 把 LoRA 注入模型
5. 跑一次 forward
```

---

## 🔍 Step 1：加载模型

```python
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
```

你加载的是一个 **完整的大模型**。

此时：

* 所有参数都 requires_grad=True
* 训练会更新全部权重

---

## 🔍 Step 2：定义 LoRA 配置

```python
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
)
```

每个参数是什么意思？

### r

低秩维度。

原本一个矩阵是：

```
W ∈ R^{d x k}
```

LoRA 把更新写成：

```
ΔW = A B
```

其中：

* A ∈ R^{d x r}
* B ∈ R^{r x k}
* r ≪ d,k

你设置 r=8，就是 rank=8。

---

### lora_alpha

缩放系数。

实际更新是：

```
W_new = W + (alpha / r) * A B
```

---

### target_modules

这是最关键的。

它告诉 PEFT：

> 哪些层的 weight 要插入 LoRA adapter。

比如 Transformer 里 attention projection 层：

```
q_proj
v_proj
```

你其实是在说：

> 只对 attention 里的 Q 和 V 线性层做低秩更新。

---

## 🔍 Step 3：注入 LoRA

```python
model = get_peft_model(model, lora_config)
```

这一步发生了什么？

PEFT 做了三件事：

1. 冻结原始权重（requires_grad=False）
2. 在 target_modules 里插入 LoRA adapter
3. 只让 LoRA 参数 requires_grad=True

---

## 🔍 Step 4：print_trainable_parameters()

输出：

```
trainable params: 147,456
all params: 82,060,032
trainable%: 0.1797
```

这说明：

* 你只训练了 0.18% 参数
* 这就是 PEFT 的核心

---

## 🔍 Step 5：Forward

```python
out = model(**inputs)
```

这一步只是：

* 做一次前向传播
* 没有 backward
* 没有更新

目的是确认：

> LoRA 注入后模型还能正常推理

---

# 实验 2：超短训练 + 生成

---

## 🎯 目标

验证 LoRA 参数真的在更新。

---

## 🧠 结构

```
1. 加载小数据
2. tokenize
3. 定义 Trainer
4. 训练若干步
5. 生成文本
```

---

## 🔍 Step 1：加载小数据

```python
ds = load_dataset("wikitext", ..., split="train[:0.2%]")
```

只是为了：

* 有 token 输入
* 不追求质量

---

## 🔍 Step 2：tokenize

```python
tokenizer(..., max_length=128)
```

变成：

```
input_ids
attention_mask
```

---

## 🔍 Step 3：Trainer 做了什么？

Trainer 内部自动做：

```
for step:
    forward
    compute loss
    backward
    optimizer.step()
```

关键点：

### 这里 optimizer 只更新 LoRA 参数

因为：

```
原模型参数 requires_grad=False
```

---

## 🔍 Step 4：Loss 抖动为什么正常？

你只训练 20 步：

* 还没收敛
* batch 小
* 梯度噪声大

这不代表失败。

---

## 🔍 Step 5：生成

```python
model.generate(...)
```

这一步用的是：

```
更新后的权重
```

如果生成变化：

说明 LoRA 参数确实生效。

---

# 🔥 你现在应该能回答的问题

我问你几个检查问题：

1. LoRA 更新的是原始 W 吗？
2. LoRA 参数在哪里？
3. 为什么 trainable% 这么低？
4. 为什么 forward 不需要 labels？
5. 为什么 Trainer 会自动算 loss？

你要能回答这些，才算真正理解。

---

# 🧠 最重要的一点

你现在做的是：

> LoRA baseline 验证

而不是：

> 研究 Stage 1 layer selection

这两个是不同层级。

你必须先能 **手写 LoRA baseline**，
才能改成 FlexLoRA。

---

# 🎓 下次你自己写代码的骨架

你下次应该能写出：

```
load model
define LoraConfig
get_peft_model
print_trainable_parameters
(optional) Trainer
generate
```

In [1]:
cd /content/drive/MyDrive/Colab Notebooks/COMP9991

/content/drive/MyDrive/Colab Notebooks/COMP9991


In [2]:
pwd

'/content/drive/MyDrive/Colab Notebooks/COMP9991'

In [3]:
cd COMP9991_PEFT

/content/drive/MyDrive/Colab Notebooks/COMP9991/COMP9991_PEFT


In [4]:
pwd

'/content/drive/MyDrive/Colab Notebooks/COMP9991/COMP9991_PEFT'

In [ ]:
!git clone https://github.com/huggingface/peft.git

Cloning into 'peft'...
remote: Enumerating objects: 15082, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (380/380), done.
remote: Total 15082 (delta 381), reused 138 (delta 138), pack-reused 14564 (from 3)
Receiving objects: 100% (15082/15082), 25.04 MiB | 11.70 MiB/s, done.
Resolving deltas: 100% (10392/10392), done.
Updating files: 100% (673/673), done.


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/COMP9991/COMP9991_PEFT/peft")

import peft
print(peft.__version__)

0.18.1


# 片段 1：LoRA 挂载 + 看“可训练参数” + 跑一次 **forward（最稳）**

In [5]:
!pip -q install -U "transformers==4.41.2" "peft==0.11.1" accelerate datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.6 MB/s eta 0:00:00


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # OPT 用这个
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

text = "COMP9991 is about"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    out = model(**inputs)
print("Forward OK. Logits shape:", out.logits.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 294,912 || all params: 125,534,208 || trainable%: 0.2349
Forward OK. Logits shape: torch.Size([1, 7, 50272])


In [8]:
# ===== Quickstart B: tiny training loop (10-20 steps) =====
import torch
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

# 取一个很小的文本数据集子集（别贪多）
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:100]")

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=128,
        padding="max_length",
    )

tokenized = ds.map(tokenize_fn, batched=True, remove_columns=ds.column_names)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

args = TrainingArguments(
    output_dir="./tmp_lora_quickstart",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    max_steps=20,                 # 关键：就跑 20 步
    learning_rate=2e-4,
    logging_steps=5,
    save_steps=0,
    report_to=[],
    fp16=torch.cuda.is_available(),  # 有GPU就开
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    data_collator=data_collator,
)

trainer.train()

# 训练后生成一句，看看“能生成”即可（别期待质量）
prompt = "LoRA makes fine-tuning"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
gen = model.generate(**inputs, max_new_tokens=30, do_sample=True, top_p=0.9)
print(tokenizer.decode(gen[0], skip_special_tokens=True))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss
5,4.741600
10,5.788500
15,5.446500
20,5.925600


LoRA makes fine-tuning of its mobile phone
LRA says it has made "a significant progress" in its mobile phone market, making it the industry's largest market.


In [9]:
for name, module in model.named_modules():
    if "proj" in name:
        print(name)

base_model.model.model.decoder.layers.0.self_attn.k_proj
base_model.model.model.decoder.layers.0.self_attn.v_proj
base_model.model.model.decoder.layers.0.self_attn.v_proj.base_layer
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_dropout
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_dropout.default
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A.default
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B.default
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_embedding_A
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_embedding_B
base_model.model.model.decoder.layers.0.self_attn.q_proj
base_model.model.model.decoder.layers.0.self_attn.q_proj.base_layer
base_model.model.model.decoder.layers.0.self_attn.q_proj.lora_dropout
base_model.model.model.decoder.layers.0.self_attn.q_pro